<a href="https://colab.research.google.com/github/bdvir/hungry-geese/blob/master/handyRL%40optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# Install HandyRL @ Optuna frameworks

In [ ]:
cd =  "drive/MyDrive/HandyRL-master/"

In [ ]:
!git clone https://github.com/DeNA/HandyRL.git


In [ ]:
!git clone https://github.com/bdvir/hungry-geese.git

In [ ]:
%cd "drive/MyDrive/HandyRL-master/"
!pip install -r requirements.txt
!pip install -r requirements-dev.txt
!pip install optuna
!pip install kaggle_environments

/content/drive/MyDrive/HandyRL-master
     |████████████████████████████████| 15.3 MB 170 kB/s 
     |████████████████████████████████| 296 kB 56.1 MB/s 
     |████████████████████████████████| 636 kB 42.0 MB/s 
     |████████████████████████████████| 804.1 MB 1.7 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

Ignoring atomicwrites: markers 'python_version >= "3.6" and python_full_version < "3.0.0" and sys_platform == "win32" or sys_platform == "win32" and python_version >= "3.6" and python_full_version >= "3.4.0"' don't match your environment
Ignoring colorama: markers 'python_version >= "3.6" and python_full_version < "3.0.0" and sys_platform == "win32" or sys_platform == "win32" and python_version >= "3.6" and python_full_version >= "3.5.0"' don't match your environment
     |████████████████████████████████| 49 kB 2.8 MB/s 
     |████████████████████████████████| 147 kB 8.4 MB/s 
     |████████████████████████████████| 178 kB 9.2 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 108 kB 10.7 MB/s 
     |████████████████████████████████| 40 kB 4.9 MB/s 
     |████████████████████████████████| 106 kB 9.4 MB/s 
     |████████████████████████████████| 280 kB 10.7 MB/s 
     |████████████████████████████████| 61 kB 6.1 MB/s 
     |███████████

In [ ]:
import collections

import kaggle_environments
from kaggle_environments import evaluate, make, utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

import subprocess
import yaml
import io
import optuna
import pickle

Loading environment football failed: No module named 'gfootball'



# Subprocess function

In [ ]:
def execute(cmd):
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line 
    popen.stdout.close()
    return_code = popen.wait()
    if return_code:
        raise subprocess.CalledProcessError(return_code, cmd)

# check subprocess function
the code shown below is just to valid the subprocess function

In [ ]:
# Example
with open("config.yaml") as stream:
    data = yaml.safe_load(stream)

# change train configurations
data['train_args']['value_target'] = 'TD'

data['train_args']['policy_target'] = 'TD'

# data['train_args']['epochs'] = 400
# data['train_args']['minimum_episodes'] = 0
# data['train_args']['maximum_episodes'] = 90
data['env_args']['env'] = 'HungryGeese'
data['env_args']['source'] = 'handyrl.envs.kaggle.hungry_geese'

with io.open('config.yaml', 'w', encoding='utf8') as outfile:
  yaml.dump(data, outfile, default_flow_style=False, allow_unicode=True)

for path in execute(["python3", "main.py", '--train']):
    print(path, end="")

    if 'loss' in path:
      print(path.split()[5][6::])
      # scores = one_on_one_with_two_simple(list_agents)[1,0] / 50
      hard = hard_one_on_one_with_two_simple(list_agents)[0,1]
      easy = one_on_one_with_two_simple(list_agents)[0,1]
      print([hard, easy])
      print((hard + easy)/15)
    if 'finished training' in path:
      break
      # if epoch == "30":
        # break


# Compare Agent with smart_reinforcement_learning Agent

In [ ]:
def hard_one_on_one_with_two_simple(agents):
    n_agents = len(agents)

    scores = np.zeros((n_agents, n_agents), dtype=np.int)

    print("Simulation of battles. It can take some time...")

    for ind_1 in range(n_agents):
        for ind_2 in range(ind_1 + 1, n_agents):
            print(
                f"LOG: {agents[ind_1]} vs {agents[ind_2]} vs 2 X simple_toward",
                end="\r"
            )

            current_score = evaluate(
                "hungry_geese",
                [
                    agents[ind_1],
                    agents[ind_2],
                    "hungry-geese-master/Kaggle Agents/simple_toward.py",
                    "hungry-geese-master/Kaggle Agents/simple_toward.py",
                ],
                num_episodes=100,
            )

            episode_winners = np.argmax(current_score, axis=1)
            episode_winner_counts = collections.Counter(episode_winners)

            scores[ind_1, ind_2] = episode_winner_counts.get(0, 0)
            scores[ind_2, ind_1] = episode_winner_counts.get(1, 0)

        print()

    return scores

In [ ]:
list_names = [
    "hungry-geese-master/Kaggle Agents/smart_reinforcement_learning2",
    "hungry-geese-master/Kaggle Agents/smart_reinforcement_learning",
]
list_agents = [agent_name + ".py" for agent_name in list_names]

# hard_one_on_one_with_two_simple
try:
  scores = one_on_one_with_two_simple(list_agents)
except TypeError:
  print(TypeError)


Simulation of battles. It can take some time...
LOG: hungry-geese-master/Kaggle Agents/smart_reinforcement_learning2.py vs hungry-geese-master/Kaggle Agents/smart_reinforcement_learning.py vs 2 X simple_toward



In [ ]:
hard = hard_one_on_one_with_two_simple(list_agents)[0,1]
print((hard)/100)

Simulation of battles. It can take some time...
LOG: hungry-geese-master/Kaggle Agents/smart_reinforcement_learning2.py vs hungry-geese-master/Kaggle Agents/smart_reinforcement_learning.py vs 2 X simple_toward

Simulation of battles. It can take some time...


[0, 0]
0.0


# Using optuna framework to find the best Hyperparameters

In [ ]:
# %cd HandyRL

# !python main.py --train


# read config file
with open("config.yaml") as stream:
    data = yaml.safe_load(stream)


data['train_args']['epochs'] = 60

global score
def objective(trial):


    data['train_args']['batch_size'] = trial.suggest_int('batch_size', 50, 300, log=True)
    data['train_args']['forward_steps'] = trial.suggest_int('forward_steps', 10, 50, log=True)
    data['train_args']['gamma'] = trial.suggest_float('gamma', 0.5, 0.95, log=True)
    data['train_args']['entropy_regularization'] = trial.suggest_float('entropy_regularization', 1.0e-3, 1.0e-1, log=True)
    data['train_args']['entropy_regularization_decay'] = trial.suggest_float('entropy_regularization_decay', 0.05, 0.5, log=True)
    # data['train_args']['policy_target'] = trial.suggest_categorical('policy_target', ['UPGO' 'VTRACE' 'TD' 'MC'])
    # data['train_args']['value_target'] = trial.suggest_categorical('value_target', ['VTRACE' 'TD' 'MC'])
    data['train_args']['lambda'] = trial.suggest_float('lambda', 0.05, 0.5, log=True)



    with io.open('config.yaml', 'w', encoding='utf8') as outfile:
      yaml.dump(data, outfile, default_flow_style=False, allow_unicode=True)

    for path in execute(["python3", "main.py", '--train']):
      print(path, end="")
      # win_rate = path.split()[3]
        
      if 'epoch' in path:
        epoch = path.split()[1]

      if 'loss' in path:

        loss = path.split()[5][6::]
        print(f"loss {float(loss)} epoch {float(epoch)}")


        trial.report(float(loss)*-1, float(epoch))

        # Handle pruning based on the intermediate value.
      if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
        try:
          hard = hard_one_on_one_with_two_simple(list_agents)[0,1]
          print([hard])
          score = (hard)/100
        except TypeError:
          print(TypeError)
          print(score)
        break
      if 'finished training' in path:
        try:
          hard = hard_one_on_one_with_two_simple(list_agents)[0,1]
          print([hard])
          score = (hard)/100
        except TypeError:
          print(TypeError)
          print(score)
        break

    return score

In [ ]:
study = optuna.create_study(study_name="optmize_2",
                            storage='sqlite:///optuna2.db',
                            load_if_exists=True,
                            direction='maximize',
                            sampler=optuna.samplers.RandomSampler(),
                            pruner=optuna.pruners.PercentilePruner(60.0, n_startup_trials=3, n_warmup_steps=10,n_min_trials = 10),
)
study.optimize(objective, n_trials=30)

[I 2021-07-29 00:42:58,392] Using an existing study with name 'optmize_2' instead of creating a new one.


{'env_args': {'env': 'HungryGeese', 'source': 'handyrl.envs.kaggle.hungry_geese'}, 'train_args': {'batch_size': 278, 'compress_steps': 4, 'entropy_regularization': 0.00103331354447033, 'entropy_regularization_decay': 0.07586031215043337, 'epochs': 60, 'eval': {'opponent': ['random']}, 'eval_rate': 0.1, 'forward_steps': 24, 'gamma': 0.7415053829522315, 'lambda': 0.05489810161818667, 'maximum_episodes': 100000, 'minimum_episodes': 400, 'num_batchers': 2, 'observation': False, 'policy_target': 'TD', 'restart_epoch': 0, 'seed': 0, 'turn_based_training': True, 'update_episodes': 200, 'value_target': 'TD', 'worker': {'num_parallel': 6}}, 'worker_args': {'num_parallel': 8, 'server_address': ''}}
Loading environment football failed: No module named 'gfootball'
waiting training
started gather 0
started server
100 200 300 400 start sender
start receiver 0
start receiver 1
started training
500 600 
epoch 0
win rate = 0.931 (249.5 / 268)
generation stats = 0.000 +- 0.705
loss = p:-0.001 v:0.381 en

[I 2021-07-29 01:38:46,618] Trial 11 finished with value: 0.02 and parameters: {'batch_size': 278, 'forward_steps': 24, 'gamma': 0.7415053829522315, 'entropy_regularization': 0.00103331354447033, 'entropy_regularization_decay': 0.07586031215043337, 'lambda': 0.05489810161818667}. Best is trial 5 with value: 0.03.




[2]
{'env_args': {'env': 'HungryGeese', 'source': 'handyrl.envs.kaggle.hungry_geese'}, 'train_args': {'batch_size': 66, 'compress_steps': 4, 'entropy_regularization': 0.06564062526494233, 'entropy_regularization_decay': 0.30612876457574995, 'epochs': 60, 'eval': {'opponent': ['random']}, 'eval_rate': 0.1, 'forward_steps': 28, 'gamma': 0.67248200837052, 'lambda': 0.0824260801482573, 'maximum_episodes': 100000, 'minimum_episodes': 400, 'num_batchers': 2, 'observation': False, 'policy_target': 'TD', 'restart_epoch': 0, 'seed': 0, 'turn_based_training': True, 'update_episodes': 200, 'value_target': 'TD', 'worker': {'num_parallel': 6}}, 'worker_args': {'num_parallel': 8, 'server_address': ''}}
Loading environment football failed: No module named 'gfootball'
waiting training
started gather 0
started server
100 200 300 400 start sender
start receiver 0
start receiver 1
started training
500 600 
epoch 0
win rate = 0.939 (252.5 / 269)
generation stats = 0.000 +- 0.705
loss = p:0.031 v:0.223 e

[I 2021-07-29 02:01:45,199] Trial 12 finished with value: 0.02 and parameters: {'batch_size': 66, 'forward_steps': 28, 'gamma': 0.67248200837052, 'entropy_regularization': 0.06564062526494233, 'entropy_regularization_decay': 0.30612876457574995, 'lambda': 0.0824260801482573}. Best is trial 5 with value: 0.03.


...



[0]
{'env_args': {'env': 'HungryGeese', 'source': 'handyrl.envs.kaggle.hungry_geese'}, 'train_args': {'batch_size': 179, 'compress_steps': 4, 'entropy_regularization': 0.007207488795832191, 'entropy_regularization_decay': 0.059893156203357605, 'epochs': 60, 'eval': {'opponent': ['random']}, 'eval_rate': 0.1, 'forward_steps': 13, 'gamma': 0.8622539918552627, 'lambda': 0.0968239364523544, 'maximum_episodes': 100000, 'minimum_episodes': 400, 'num_batchers': 2, 'observation': False, 'policy_target': 'TD', 'restart_epoch': 0, 'seed': 0, 'turn_based_training': True, 'update_episodes': 200, 'value_target': 'TD', 'worker': {'num_parallel': 6}}, 'worker_args': {'num_parallel': 8, 'server_address': ''}}
Loading environment football failed: No module named 'gfootball'
waiting training
started gather 0
started server
100 200 300 400 start sender
start receiver 0
start receiver 1
started training
500 600 
epoch 0
win rate = 0.919 (248.0 / 270)
generation stats = 0.000 +- 0.707
loss = p:0.060 v:0.

[I 2021-07-29 04:22:50,756] Trial 16 pruned. 


{'env_args': {'env': 'HungryGeese', 'source': 'handyrl.envs.kaggle.hungry_geese'}, 'train_args': {'batch_size': 207, 'compress_steps': 4, 'entropy_regularization': 0.004817964848738254, 'entropy_regularization_decay': 0.060632934534728705, 'epochs': 60, 'eval': {'opponent': ['random']}, 'eval_rate': 0.1, 'forward_steps': 11, 'gamma': 0.6461863750716504, 'lambda': 0.18613652960346397, 'maximum_episodes': 100000, 'minimum_episodes': 400, 'num_batchers': 2, 'observation': False, 'policy_target': 'TD', 'restart_epoch': 0, 'seed': 0, 'turn_based_training': True, 'update_episodes': 200, 'value_target': 'TD', 'worker': {'num_parallel': 6}}, 'worker_args': {'num_parallel': 8, 'server_address': ''}}
Loading environment football failed: No module named 'gfootball'
waiting training
started gather 0
started server
100 200 300 400 start sender
start receiver 0
start receiver 1
started training
500 600 
epoch 0
win rate = 0.952 (256.2 / 269)
generation stats = 0.000 +- 0.708
loss = p:-0.004 v:0.390 

[I 2021-07-29 05:55:54,051] Trial 17 finished with value: 0.0 and parameters: {'batch_size': 207, 'forward_steps': 11, 'gamma': 0.6461863750716504, 'entropy_regularization': 0.004817964848738254, 'entropy_regularization_decay': 0.060632934534728705, 'lambda': 0.18613652960346397}. Best is trial 5 with value: 0.03.




[0]
{'env_args': {'env': 'HungryGeese', 'source': 'handyrl.envs.kaggle.hungry_geese'}, 'train_args': {'batch_size': 99, 'compress_steps': 4, 'entropy_regularization': 0.006557488821613261, 'entropy_regularization_decay': 0.07545842089605802, 'epochs': 60, 'eval': {'opponent': ['random']}, 'eval_rate': 0.1, 'forward_steps': 13, 'gamma': 0.9414135582579298, 'lambda': 0.12282814111990377, 'maximum_episodes': 100000, 'minimum_episodes': 400, 'num_batchers': 2, 'observation': False, 'policy_target': 'TD', 'restart_epoch': 0, 'seed': 0, 'turn_based_training': True, 'update_episodes': 200, 'value_target': 'TD', 'worker': {'num_parallel': 6}}, 'worker_args': {'num_parallel': 8, 'server_address': ''}}
Loading environment football failed: No module named 'gfootball'
waiting training
started gather 0
started server
100 200 300 400 start sender
start receiver 0
start receiver 1
started training
500 600 
epoch 0
win rate = 0.917 (245.8 / 268)
generation stats = 0.000 +- 0.708
loss = p:-0.019 v:0.

# Plotting optuna graphs

In [ ]:
study = optuna.create_study(study_name="optmize_2",
                            storage='sqlite:///optuna2.db',
                            load_if_exists=True,
                            direction='maximize',
                            sampler=optuna.samplers.RandomSampler(),
                            pruner=optuna.pruners.PercentilePruner(60.0, n_startup_trials=3, n_warmup_steps=10,n_min_trials = 10),
)

[I 2021-07-29 08:43:24,804] Using an existing study with name 'optmize_2' instead of creating a new one.


In [ ]:
study.best_trial

FrozenTrial(number=5, values=[0.03], datetime_start=datetime.datetime(2021, 7, 28, 19, 11, 45, 606595), datetime_complete=datetime.datetime(2021, 7, 28, 20, 12, 1, 163455), params={'batch_size': 194, 'entropy_regularization': 0.021083941325056196, 'entropy_regularization_decay': 0.20720924104573812, 'forward_steps': 23, 'gamma': 0.5029538438013005, 'lambda': 0.4599298176495907}, distributions={'batch_size': IntLogUniformDistribution(high=300, low=50, step=1), 'entropy_regularization': LogUniformDistribution(high=0.1, low=0.001), 'entropy_regularization_decay': LogUniformDistribution(high=0.5, low=0.05), 'forward_steps': IntLogUniformDistribution(high=50, low=10, step=1), 'gamma': LogUniformDistribution(high=0.95, low=0.5), 'lambda': LogUniformDistribution(high=0.5, low=0.05)}, user_attrs={}, system_attrs={}, intermediate_values={0: -0.136, 1: -0.146, 2: -0.129, 3: -0.139, 4: -0.132, 5: -0.129, 6: -0.142, 7: -0.133, 8: -0.13, 9: -0.13, 10: -0.136, 11: -0.142, 12: -0.139, 13: -0.139, 14:

In [ ]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

plot_optimization_history(study)

In [ ]:
plot_intermediate_values(study)

In [ ]:
        "batch_size": trial.suggest_int('batch_size', 4, 60),
        "gamma": trial.suggest_float('gamma', 0.5, 0.9),
        "lambda": trial.suggest_float('lambda', 0.5, 0.9),

In [ ]:
plot_contour(study, params=["batch_size", "gamma", "lambda"])